In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Normalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

2024-10-28 15:57:14.609696: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730102234.624165  238483 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730102234.628274  238483 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 15:57:14.644429: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = np.loadtxt("./bmi.csv",delimiter=",",dtype=str)
header_row = data[0]

x_data_dirty = data[1:,0:3]
y_data = [float(row) for row in data[1:,3]]
x_data = [[float(item) if item not in ['Male', 'Female'] else (0 if item == 'Male' else 1) for item in item_row] for item_row in x_data_dirty] # Dont ask me how I managed to do this, because my future me wouldn't even know what this is even mean

for i,row in enumerate(header_row):
    print(f"{i}: {row}")


0: Gender
1: Height
2: Weight
3: Index


In [3]:
m = len(data)-1
n = len(header_row) -1
categories = len(list(set(y_data)))

x_train,x_,y_train,y_ = train_test_split(x_data,y_data,test_size=0.4,random_state=1)
x_cv,x_test,y_cv,y_test = train_test_split(x_,y_,test_size=0.50,random_state=1)

del x_,y_
x_cv_normalizer = Normalization(axis=-1)
x_cv_normalizer.adapt(np.array(x_train))
x_cv_normalized = x_cv_normalizer(np.array(x_cv))

I0000 00:00:1730102237.169120  238483 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2179 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
model1 = Sequential([
    Input(shape=(n,)),
        Dense(units=12,activation='relu'),
        Dense(units=12,activation='relu'),
        Dense(units=categories,activation='softmax')
])
model2 = Sequential([
    Input(shape=(n,)),
        Dense(units=12,activation='relu'),
        Dense(units=12,activation='relu'),
        Dense(units=categories,activation='linear')
])
model3 = Sequential([
    Input(shape=(n,)),
        Dense(units=12,activation='relu'),
        Dense(units=6,activation='relu'),
        Dense(units=categories,activation='softmax')
])
model4 = Sequential([
        Input(shape=(n,)),
        Dense(units=6,activation='relu'),
        Dense(units=12,activation='relu'),
        Dense(units=categories,activation='softmax')
])
model5 = Sequential([
    Input(shape=(n,)),
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=categories, activation='softmax')
])

nn_models = [model1, model2, model3,model4,model5]

x_train_normalizer = Normalization(axis=-1)
x_train_normalizer.adapt(np.array(x_train))
x_train_normalized = x_train_normalizer(np.array(x_train))

model_train_scores = []
model_cv_scores = []
for row, model in enumerate(nn_models):
        model.compile(optimizer=Adam(learning_rate=0.01), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
        report = model.fit(x_train_normalized, np.array(y_train), epochs=100, batch_size=32, verbose=1)
        print(f"Model {row+1} with training data score: {report.history['accuracy']}")
        y_cv_predictions = model.predict(x_cv_normalized)
        num_predictions = len(y_cv_predictions)
        y_cv_hat = np.argmax(y_cv_predictions, axis=1)
        misclassified = np.mean(y_cv_hat != y_cv)
        print(f"Model {row+1} misclassified data: {misclassified}")
        model_train_scores.append(np.max(report.history['accuracy']))
        model_cv_scores.append(1-misclassified)
        
print(model_train_scores)
print(model_cv_scores)

Epoch 1/100


/home/dan/anaconda3/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
I0000 00:00:1730102238.755698  238617 service.cc:148] XLA service 0x785c08009470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730102238.755722  238617 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-10-28 15:57:18.774506: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730102238.845343  238617 cuda_dnn.cc:529] Loaded cuDNN version 90300


 2/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0391 - loss: 1.8843

I0000 00:00:1730102239.494053  238617 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.1944 - loss: 1.8041
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5343 - loss: 1.4674
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5397 - loss: 1.1914
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.6472 - loss: 0.9242
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7529 - loss: 0.7581
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - accuracy: 0.8314 - loss: 0.6281
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8124 - loss: 0.5574
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8120 - loss: 0.5133  
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 950us/step - accuracy: 0.8561 - loss: 0.4482
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.8728 - loss: 0.3771
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8712 - loss: 0.3847
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 

In [9]:
# Train the best model with Test Data

x_test_normalizer = Normalization(axis=-1)
x_test_normalizer.adapt(np.array(x_test))
x_test_normalized = x_test_normalizer(x_test)

best_model_index = (model_cv_scores.index(max(model_cv_scores)))
best_model = nn_models[best_model_index]
print(f"Best Model: Model {best_model_index+1}")

y_test_predictions = model.predict(x_test_normalized)
num_predictions = len(y_test_predictions)
y_test_hat = np.argmax(y_test_predictions, axis=1)
misclassified = np.mean(y_test_hat != y_test)
print(f"Model {row+1} misclassified data: {misclassified}")

print(f"Model {best_model_index+1} with test data score: {1-misclassified}")

Best Model: Model 4
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step
Model 5 misclassified data: 0.04
Model 4 with test data score: 0.96
